In [3]:
def findPrime(x):
    prime=[2, 3] #
    notPrime=[]
    numbers=[2,3]
    exclude=4 #done to exclude 0 and 1 as they are unique numbers, exclude 2 and 3 as to get the list started.
    for i in range(x-exclude+1):
        num=i+exclude
        status=True
        
        for j in prime:
            if num%j==0:
                status=False
        numbers.append(num)
        if status:
            prime.append(num)
        else:
            notPrime.append(num)
    return(prime, notPrime, numbers)

number=10000
prime, notPrime,numbers=findPrime(number)
#print(prime, notPrime)

        

In [4]:
import numpy as np

label=np.zeros((number-1,1))
print(label)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]


In [5]:
for i in prime:
    label[i-2]=1
#print(label)

from keras.utils import np_utils
label_final = np_utils.to_categorical(np.array(label), 2)
print(label)
print(label_final)

[[ 1.]
 [ 1.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [14]:
data=np.zeros((number-1,1,len(str(number))))
for i in range(number-1):
    num=i+2
    for k in range(len(str(num))):
        data[i][0][len(str(number))-k-1]=int(str(num)[len(str(num))-k-1])
#print(data)
#data_final = np_utils.to_categorical(np.array(data), number+1)
print(data)

[[[ 0.  0.  0.  0.  2.]]

 [[ 0.  0.  0.  0.  3.]]

 [[ 0.  0.  0.  0.  4.]]

 ..., 
 [[ 0.  9.  9.  9.  8.]]

 [[ 0.  9.  9.  9.  9.]]

 [[ 1.  0.  0.  0.  0.]]]


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label_final, test_size=0.2, random_state=42)

In [8]:
from sklearn.ensemble import AdaBoostClassifier








In [51]:
#neural network(s):
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

print(X_train.shape)

Prime_model = Sequential()
Prime_model.add(Conv1D(filters=1024, kernel_size=2, padding='same', activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))
#Prime_model.add(MaxPooling1D(pool_size=2))
Prime_model.add(Conv1D(filters=2048, kernel_size=2, padding='same', activation='relu'))
#Prime_model.add(MaxPooling1D(pool_size=2))
Prime_model.add(Conv1D(filters=4096, kernel_size=2, padding='same', activation='relu'))
#Prime_model.add(MaxPooling1D(pool_size=4))
#Prime_model.add(AveragePooling1D(pool_size=(2, 2), strides=None, padding='valid'))
Prime_model.add(Dropout(.3))
Prime_model.add(Flatten())
Prime_model.add(Dense(2, activation='softmax'))

Prime_model.summary()

(7999, 1, 5)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 1, 1024)           11264     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 1, 2048)           4196352   
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 1, 4096)           16781312  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 4096)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 8194      
Total params: 20,997,122.0
Trainable params: 20,997,122.0
Non-trainable params: 0.0
_____________________________________________

In [52]:
#compile network:
Prime_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#running network:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 15

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='D:\Machine Learning\LearningPrime/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

Prime_model.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          epochs=epochs, batch_size=7, callbacks=[checkpointer], verbose=1)

Train on 7999 samples, validate on 2000 samples
Epoch 1/15
 147/7999 [..............................] - ETA: 1846s - loss: 2.7601 - acc: 0.8095

In [26]:
#prediction:
#load model
Prime_model.load_weights('D:\Machine Learning\LearningPrime/weights.best.from_scratch.hdf5')

In [41]:
def num(x, number):
    zeros=np.zeros(len(str(number)))
    for k in range((len(str(x)))):
        zeros[len(str(number))-k-1]=int(str(x)[len(str(x))-k-1])
    return(np.array([zeros]))


print(num(10, number))

[[ 0.  0.  0.  1.  0.]]


In [42]:
def predPrime(x):
    #Prime_model.predict(np.array([num(x, number)]))
    #print(Prime_model.predict(np.array([num(x, number)]))[0])
    return(Prime_model.predict(np.array([num(x, number)]))[0][0],Prime_model.predict(np.array([num(x, number)]))[0][1])

In [43]:
for i in range(20):
    probNotPrime, probPrime= predPrime(prime[i])
    print('i=', prime[i], '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)

for i in range(20):
    probNotPrime, probPrime= predPrime(prime[len(prime)-20+i])
    print('i=', prime[len(prime)-20+i], '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)

i= 2   probNotPrime= 0.791068   probPrime= 0.208932
i= 3   probNotPrime= 0.783379   probPrime= 0.216621
i= 5   probNotPrime= 0.790333   probPrime= 0.209667
i= 7   probNotPrime= 0.798508   probPrime= 0.201492
i= 11   probNotPrime= 0.78985   probPrime= 0.21015
i= 13   probNotPrime= 0.786353   probPrime= 0.213647
i= 17   probNotPrime= 0.801822   probPrime= 0.198178
i= 19   probNotPrime= 0.809598   probPrime= 0.190402
i= 23   probNotPrime= 0.790224   probPrime= 0.209776
i= 29   probNotPrime= 0.813001   probPrime= 0.186999
i= 31   probNotPrime= 0.786742   probPrime= 0.213258
i= 37   probNotPrime= 0.809656   probPrime= 0.190344
i= 41   probNotPrime= 0.791054   probPrime= 0.208946
i= 43   probNotPrime= 0.799487   probPrime= 0.200513
i= 47   probNotPrime= 0.815608   probPrime= 0.184392
i= 53   probNotPrime= 0.805722   probPrime= 0.194278
i= 59   probNotPrime= 0.828881   probPrime= 0.171119
i= 61   probNotPrime= 0.803721   probPrime= 0.196279
i= 67   probNotPrime= 0.827077   probPrime= 0.172923

In [44]:
#for i in notPrime:
#    probNotPrime, probPrime= predPrime(i)
#    print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
    
    
for i in range(20):
    probNotPrime, probPrime= predPrime(notPrime[i])
    print('i=', notPrime[i], '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)

for i in range(20):
    probNotPrime, probPrime= predPrime(notPrime[len(notPrime)-20+i])
    print('i=', notPrime[len(notPrime)-20+i], '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)

i= 4   probNotPrime= 0.786336   probPrime= 0.213664
i= 6   probNotPrime= 0.794451   probPrime= 0.205549
i= 8   probNotPrime= 0.802505   probPrime= 0.197495
i= 9   probNotPrime= 0.806416   probPrime= 0.193584
i= 10   probNotPrime= 0.999996   probPrime= 3.68429e-06
i= 12   probNotPrime= 0.783288   probPrime= 0.216712
i= 14   probNotPrime= 0.790009   probPrime= 0.209991
i= 15   probNotPrime= 0.793785   probPrime= 0.206215
i= 16   probNotPrime= 0.797814   probPrime= 0.202186
i= 18   probNotPrime= 0.805748   probPrime= 0.194252
i= 20   probNotPrime= 0.999998   probPrime= 2.31241e-06
i= 21   probNotPrime= 0.783329   probPrime= 0.216671
i= 22   probNotPrime= 0.786548   probPrime= 0.213452
i= 24   probNotPrime= 0.793997   probPrime= 0.206003
i= 25   probNotPrime= 0.79772   probPrime= 0.20228
i= 26   probNotPrime= 0.801392   probPrime= 0.198608
i= 27   probNotPrime= 0.805078   probPrime= 0.194922
i= 28   probNotPrime= 0.809046   probPrime= 0.190954
i= 30   probNotPrime= 0.999999   probPrime= 1.

In [45]:
import math


In [46]:
#data1D=[]
notPrimeArray=[]

for i in numbers:
    #data1D.append(int(i[0]))
    #print (i)
    #print (int(i[0]))
    #print (2)
    #probNotPrime, probPrime=predPrime(int(i[0]))
    probNotPrime, probPrime=predPrime(i)
    notPrimeArray.append(probNotPrime)
    

#print(numbers)
#print(notPrimeArray)
    

In [47]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

labelArray=[]
for i in label:
    #print (i[0])
    labelArray.append(i[0])

notPrimeArray2D=[] 
for i in notPrimeArray:
    notPrimeArray2D.append([i])
    
#print (labelArray[0])    

X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(notPrimeArray2D, labelArray, test_size=0.2, random_state=42)

clf=AdaBoostClassifier(n_estimators=50)

print (len(X_train_boost), len(y_train_boost))

#print (X_train_boost)
#print (y_train_boost)

clf.fit(X_train_boost, y_train_boost)




7999 7999


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [48]:
def boostpredict(x):
    return(clf.predict(x))

In [49]:
accuracyPrime=0
for i in prime:
    probNotPrime, probPrime= predPrime(i)
    #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
    result=boostpredict(probNotPrime)
    #print('i=', i, '  probNotPrime=', result)
    if result==[1.]:
        accuracyPrime+=1

print(accuracyPrime/len(prime))


0.009764035801464606


In [50]:
#for i in notPrime:
#    probNotPrime, probPrime= predPrime(i)
#    #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
#    print('i=', i, '  probNotPrime=', boostpredict(probNotPrime))
    
    
accuracyNotPrime=0
for i in notPrime:
    probNotPrime, probPrime= predPrime(i)
    #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
    result=boostpredict(probNotPrime)
    #print('i=', i, '  probNotPrime=', result)
    if result==[0.]:
        accuracyNotPrime+=1

print(accuracyNotPrime/len(notPrime))

0.9990877993158495


In [330]:
def flatten_list(a, result=None):
    """Flattens a nested list.

        >>> flatten_list([ [1, 2, [3, 4] ], [5, 6], 7])
        [1, 2, 3, 4, 5, 6, 7]
    """
    print(a)
    if result is None:
        result = []

    for x in a:
        if isinstance(x, list):
            flatten_list(x, result)
        else:
            result.append(x)

    return result

a=[ [1, 2, [3, 4] ], [5, 6], 7]
print(flatten_list(a, result=None))

[[1, 2, [3, 4]], [5, 6], 7]
[1, 2, [3, 4]]
[3, 4]
[5, 6]
[1, 2, 3, 4, 5, 6, 7]


In [360]:
print (np.array([num(1, number)]))

[[ 0.  1.  0. ...,  0.  0.  0.]]


In [408]:
print(data)
print (predPrime(2))
probNotPrime, probPrime= predPrime(2)

print (probNotPrime*10)
a= (math.floor(probNotPrime*10))
b=math.floor(probNotPrime*100-a*10)
c=math.floor(probNotPrime*1000-(a*100+b*10))
d=math.floor(probNotPrime*10000-(a*1000+b*100+c*10))
e=math.floor(probNotPrime*100000-d*10)
f=math.floor(probNotPrime*1000000-e*10)
g=math.floor(probNotPrime*10000000-f*10)
h=math.floor(probNotPrime*100000000-g*10)
i=math.floor(probNotPrime*1000000000-h*10)
j=math.floor(probNotPrime*10000000000-i*10)

print ('a=', a)
print ('b=', b)
print ('c=', c)
print ('d=', d)
#print ('e=', e)

#print ('f=', f)
#print ('g=', g)
#print ('h=', h)
#print ('i=', i)
#print ('j=', j)

def digits(x,y):
    result=([math.floor(x*10)])
    if y>0:
        return(result.append(digits(x*10-math.floor(x*10),y-1)))
    else:
        return(result)
    
print (digits(probNotPrime, 1))
#    
#def digitsVect(x):

#for i in data:
    

[[  2.00000000e+00]
 [  3.00000000e+00]
 [  4.00000000e+00]
 ..., 
 [  4.99800000e+03]
 [  4.99900000e+03]
 [  5.00000000e+03]]
(0.84858251, 0.15141749)
8.4858250618
a= 8
b= 4
c= 8
d= 5
None
